# Prepare Data for Training with Remapped Categories

In [69]:
import os
import random

import numpy as np
import pandas as pd
from scipy.stats import mode
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

## Prepare transaction data

In [70]:
# Retrieve indices of such customers
transactions = pd.read_csv('original_data/customer_transaction_data.csv')
transactions.sample(5)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
98902,2012-04-08,212,45527,1,89.05,-78.01,0.0
98152,2012-04-08,63,45010,1,56.64,-56.99,0.0
102789,2012-04-11,1352,57777,1,99.38,0.00,0.0
692899,2012-11-16,604,16963,1,81.57,0.00,0.0
1086023,2013-04-06,293,28096,1,37.40,0.00,0.0


In [71]:
transactions.drop('date', axis=1, inplace=True)

In [72]:
transactions['original_price'] = transactions.selling_price - transactions.other_discount - transactions.coupon_discount
transactions.original_price = transactions.original_price / transactions.quantity
transactions.coupon_discount = transactions.coupon_discount / transactions.quantity
transactions.drop(['other_discount', 'selling_price'], axis=1, inplace=True)

In [73]:
transactions['coupon_used'] = transactions.coupon_discount.apply(lambda x: 1 if x != 0 else 0)
transactions.coupon_used.value_counts(normalize=True)

0    0.98393
1    0.01607
Name: coupon_used, dtype: float64

#### Modify  quantities to fit clothing store categories

In [74]:
# Scale quantity to something more realistic in a clothing store
print(f'Min quantity: {transactions.quantity.min()}')
print(f'Max quantity: {transactions.quantity.max()}')
print(f'Mean quantity: {transactions.quantity.mean()}')
print(f'Median quantity: {transactions.quantity.median()}')
print(f'Mode quantity: {transactions.quantity.mode()[0]}')

# We want min to be 1 and max to be, e.g. 23, and also for the items to be integers
scaler = MinMaxScaler((1, 23))
transactions['quantity'] = scaler.fit_transform(transactions[['quantity']])
transactions['quantity'] = transactions['quantity'].round(decimals=0).astype(np.int64)

print('\nNew values:')
print(f'Min quantity: {transactions.quantity.min()}')
print(f'Max quantity: {transactions.quantity.max()}')
print(f'Mean quantity: {transactions.quantity.mean()}')
print(f'Median quantity: {transactions.quantity.median()}')
print(f'Mode quantity: {transactions.quantity.mode()[0]}')

Min quantity: 1
Max quantity: 89638
Mean quantity: 130.6633395391396
Median quantity: 1.0
Mode quantity: 1

New values:
Min quantity: 1
Max quantity: 23
Mean quantity: 1.0317772009850774
Median quantity: 1.0
Mode quantity: 1


## Prepare customer demographics data

In [75]:
customers = pd.read_csv('original_data/customer_demographics.csv')
customers.sample(5)

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
736,1523,70+,Married,0,2,NaN,1
293,581,46-55,NaN,0,1,NaN,2
102,202,46-55,Married,0,2,NaN,6
21,39,70+,Married,0,2,NaN,4
617,1292,46-55,Married,0,2,NaN,2


In [76]:
customers.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [77]:
customers['family_size'] = customers.family_size.apply(lambda x: int(x.replace('+', '')))
customers['no_of_children'] = customers.no_of_children.apply(lambda x: int(x.replace('+', '')) if pd.notna(x) else x)
customers.loc[pd.isnull(customers.marital_status) & (customers.family_size == 1),
              'marital_status'] = 'Single'
customers.loc[pd.isnull(customers.marital_status) & ((customers.family_size - customers.no_of_children) == 1),
              'marital_status'] = 'Single'
customers.loc[pd.isnull(customers.marital_status) & ((customers.family_size - customers.no_of_children) == 2),
              'marital_status'] = 'Married'
customers.loc[pd.isnull(customers.marital_status) & pd.isnull(customers.no_of_children) & (customers.family_size == 2),
              'marital_status'] = 'Married'
customers.loc[pd.isnull(customers.no_of_children) & (customers.marital_status == 'Married') & (customers.family_size == 2),
              'no_of_children'] = 0
customers.loc[pd.isnull(customers.no_of_children) & (customers.family_size == 1), 'no_of_children'] = 0
customers.loc[pd.isnull(customers.no_of_children) & (customers.family_size == 2), 'no_of_children'] = 1
customers['no_of_children'] = customers['no_of_children'].astype(np.int64)

In [78]:
customers.drop('rented', axis=1, inplace=True)

In [79]:
len(customers)

760

#### Fill in customer demographics data for customer ids absent from customers dataframe, but present in transactions dataframe

In [80]:
ids_to_add = sorted([i for i in transactions.customer_id.unique() if i not in customers.customer_id.values])
len(ids_to_add)

822

In [81]:
# Calculate statistics for existing customers
demographics = customers.drop('customer_id', axis=1).value_counts(normalize=True).rename('count_normalized').reset_index()

# Fill in missing customer info based on the statistics
columns = demographics.drop('count_normalized', axis=1).columns
values = demographics.drop('count_normalized', axis=1).to_records()
probs = demographics['count_normalized'].values
row_num = len(ids_to_add)

new_customers = pd.DataFrame(np.random.choice(values, row_num, p=probs), columns=columns)
new_customers['customer_id'] = ids_to_add
new_customers

,age_range,marital_status,family_size,no_of_children,income_bracket,customer_id
0,46-55,Married,2,0,3,2
1,26-35,Single,1,0,4,3
2,46-55,Single,1,0,5,4
3,70+,Married,2,0,4,5
4,26-35,Married,4,2,6,9
...,...,...,...,...,...,...
817,36-45,Married,5,3,6,1570
818,36-45,Single,1,0,3,1571
819,36-45,Married,5,3,5,1575
820,46-55,Single,1,0,4,1576


In [82]:
customers = customers.append(new_customers).sort_values('customer_id').reset_index(drop=True)

In [83]:
customers['gender'] = np.random.choice(['M', 'F'], len(customers), p=[0.48, 0.52])
customers.gender.value_counts()

F    815
M    767
Name: gender, dtype: int64

In [84]:
customers

,customer_id,age_range,marital_status,family_size,no_of_children,income_bracket,gender
0,1,70+,Married,2,0,4,M
1,2,46-55,Married,2,0,3,F
2,3,26-35,Single,1,0,4,M
3,4,46-55,Single,1,0,5,M
4,5,70+,Married,2,0,4,F
...,...,...,...,...,...,...,...
1577,1578,46-55,Married,3,1,6,M
1578,1579,46-55,Single,1,0,4,M
1579,1580,26-35,Married,2,0,5,M
1580,1581,26-35,Married,3,1,1,M


## Prepare item data

In [85]:
items = pd.read_csv('original_data/item_data.csv')
items.sample(5)

,item_id,brand,brand_type,category
5529,5530,56,Local,Pharmaceutical
47321,47322,280,Established,Grocery
30273,30274,1124,Established,Grocery
69119,69120,1025,Established,Grocery
39596,39597,56,Local,Grocery


In [86]:
items.drop(['brand_type', 'brand'], axis=1, inplace=True)

In [87]:
items.sort_values(by='item_id', inplace=True)

### Find out item price and mean coupon discount for that item from transactions table

In [88]:
item_price = pd.pivot_table(
    data=transactions,
    index='item_id',
    values=['original_price', 'coupon_discount'],
    aggfunc={
        'original_price': lambda x: np.round(np.mean(x), decimals=2),
        'coupon_discount': lambda x: np.round(np.mean(x), decimals=2)
    }
)
item_price.rename(columns={'original_price': 'mean_item_price', 'coupon_discount': 'mean_coupon_discount'}, inplace=True)
item_price.sample(5)

,mean_coupon_discount,mean_item_price
item_id,,
35182,0.0,28.14
43203,0.0,202.68
15337,0.0,191.99
51481,0.0,462.70
66165,0.0,284.60


In [89]:
# Merge prices with items
items = pd.merge(items, item_price, on='item_id', how='left').reset_index(drop=True)
items.sample(5)

,item_id,category,mean_coupon_discount,mean_item_price
23303,23304,Grocery,0.00,63.76
1269,1270,Grocery,0.00,117.19
28282,28283,Pharmaceutical,0.00,89.05
28388,28389,Pharmaceutical,-31.16,403.92
42121,42122,Grocery,0.00,81.57


In [90]:
items.isnull().sum()

item_id                 0
category                0
mean_coupon_discount    3
mean_item_price         3
dtype: int64

In [91]:
items.dropna(axis='index', inplace=True)
items.isnull().sum()

item_id                 0
category                0
mean_coupon_discount    0
mean_item_price         0
dtype: int64

In [92]:
items.shape

(74063, 4)

## Category Mapping

### Merge customers, items, and transactions

In [93]:
item_categories = items.drop(['mean_item_price', 'mean_coupon_discount'], axis=1)

In [94]:
df = transactions.merge(customers, on='customer_id', how='left')
df = df.merge(item_categories, on='item_id', how='left')
df.sample(5)

,customer_id,item_id,quantity,coupon_discount,original_price,coupon_used,age_range,marital_status,family_size,no_of_children,income_bracket,gender,category
979472,858,31074,1,0.0,67.32,0,46-55,Married,2,0,2,M,Grocery
1300519,808,53891,1,0.0,110.07,0,70+,Married,2,0,4,M,Grocery
809859,444,8829,1,0.0,53.07,0,46-55,Married,2,0,2,F,Pharmaceutical
1169096,892,10547,1,0.0,67.68,0,46-55,Single,1,0,2,M,Grocery
57234,631,21970,1,0.0,49.51,0,56-70,Married,2,0,5,M,Grocery


In [95]:
df.isnull().sum()

customer_id        0
item_id            0
quantity           0
coupon_discount    0
original_price     0
coupon_used        0
age_range          0
marital_status     0
family_size        0
no_of_children     0
income_bracket     0
gender             0
category           0
dtype: int64

### Category mapping

In [96]:
# Check which items are bought mainly by people with children
df['has_children'] = df.no_of_children.apply(lambda x: 1 if x > 0 else 0)
df['no_children'] = df.no_of_children.apply(lambda x: 1 if x == 0 else 0)

agg_items = df.groupby('item_id').agg({'has_children': np.sum, 'no_children': np.sum}).reset_index()
more_with_children = agg_items.loc[(agg_items['has_children'] - agg_items['no_children']) > 0]

items_for_children = more_with_children.drop(['has_children', 'no_children'], axis=1)
items_for_children['category'] = np.random.choice(['Boys', 'Girls'], len(items_for_children))
items_for_children.reset_index(drop=True, inplace=True)

In [97]:
# Check which items are more popular with each gender
df['male'] = df.gender.apply(lambda x: 1 if x == 'M' else 0)
df['female'] = df.gender.apply(lambda x: 1 if x == 'F' else 0)

agg_items = df.groupby('item_id').agg({'male': np.sum, 'female': np.sum}).reset_index()

more_by_men = agg_items.loc[(agg_items['male'] > agg_items['female'])]
more_by_women = agg_items.loc[(agg_items['female'] >= agg_items['male'])]
# Sanity check
more_by_men_transactions = more_by_men.male.sum() + more_by_men.female.sum()
more_by_women_transactions = more_by_women.female.sum() + more_by_women.male.sum()
assert len(transactions) == more_by_women_transactions + more_by_men_transactions

items_for_men = more_by_men.loc[~(more_by_men.item_id.isin(items_for_children.item_id.values))].reset_index(drop=True)
items_for_men.drop(['male', 'female'], axis=1, inplace=True)
items_for_men['category'] = np.random.choice(['Men', 'Sport'], len(items_for_men), p=[0.7, 0.3])

items_for_women = more_by_women.loc[~(more_by_women.item_id.isin(items_for_children.item_id.values))].reset_index(drop=True)
items_for_women.drop(['male', 'female'], axis=1, inplace=True)
items_for_women['category'] = np.random.choice(['Women', 'Sport'], len(items_for_women), p=[0.8, 0.2])

In [98]:
print(f'{len(items)} items total')
print(f'{len(items_for_children)} items for children')
print(f'{len(items_for_women)} items for women')
print(f'{len(items_for_men)} items for men')
not_categorized = len(items) - (len(items_for_men) + len(items_for_women) + len(items_for_children))
print(f'{not_categorized} items not categorized')

74063 items total
16161 items for children
32961 items for women
24941 items for men
0 items not categorized


In [99]:
new_items = items_for_children.append(items_for_women).append(items_for_men)
new_items

,item_id,category
0,3,Girls
1,9,Girls
2,20,Girls
3,23,Girls
4,34,Boys
...,...,...
24936,74061,Men
24937,74062,Sport
24938,74063,Men
24939,74064,Men


In [100]:
new_items.category.value_counts()

Women    26401
Men      17392
Sport    14109
Girls     8123
Boys      8038
Name: category, dtype: int64

In [101]:
items.drop('category', axis=1, inplace=True)
items = pd.merge(new_items, items, on='item_id', how='left').sort_values('item_id').reset_index(drop=True)
items

,item_id,category,mean_coupon_discount,mean_item_price
0,1,Women,0.0,124.31
1,2,Women,0.0,35.26
2,3,Girls,0.0,56.64
3,4,Men,0.0,54.85
4,5,Women,0.0,81.57
...,...,...,...,...
74058,74062,Sport,0.0,447.03
74059,74063,Men,0.0,414.97
74060,74064,Men,0.0,414.97
74061,74065,Boys,0.0,127.88


### Merge items with coupon info

In [102]:
coupons = pd.read_csv('original_data/coupon_item_mapping.csv')
coupons.sample(5)

,coupon_id,item_id
71295,32,37319
89051,32,65529
61693,23,45222
54232,23,2612
4684,42,27465


In [103]:
coupons_items = coupons.merge(items, on='item_id', how='left')
coupons_items

,coupon_id,item_id,category,mean_coupon_discount,mean_item_price
0,105,37,Women,0.0,57.25
1,107,75,Men,0.0,38.83
2,494,76,Men,0.0,106.50
3,522,77,Men,0.0,87.27
4,518,77,Men,0.0,87.27
...,...,...,...,...,...
92658,32,69268,Sport,0.0,355.84
92659,32,68502,Boys,0.0,355.84
92660,32,68612,Girls,0.0,169.20
92661,33,71390,Boys,0.0,255.58


In [104]:
coupons_items.isnull().sum()

coupon_id               0
item_id                 0
category                8
mean_coupon_discount    8
mean_item_price         8
dtype: int64

In [105]:
coupons_items.dropna(axis=0, inplace=True)
coupons_items.shape

(92655, 5)

In [106]:
coupons_items.loc[coupons_items['mean_coupon_discount'] < 0]

,coupon_id,item_id,category,mean_coupon_discount,mean_item_price
81,643,419,Women,-3.45,74.33
131,415,907,Sport,-11.87,158.75
138,527,934,Boys,-0.71,59.63
140,417,942,Sport,-11.87,155.42
141,415,942,Sport,-11.87,155.42
...,...,...,...,...,...
92578,32,71737,Girls,-17.81,378.99
92599,33,72996,Women,-19.59,121.93
92624,32,72159,Boys,-10.39,579.95
92627,32,73170,Women,-8.90,51.89


In [107]:
coupon_stats = pd.pivot_table(
    data=coupons_items,
    index=['coupon_id'],
    values=['mean_coupon_discount', 'mean_item_price'],
    aggfunc={
        'mean_coupon_discount': lambda x: np.round(np.mean(x), decimals=2),
        'mean_item_price': lambda x: np.round(np.mean(x), decimals=2)
    }
)
coupon_stats.shape

(1116, 2)

In [108]:
coupon_stats.loc[coupon_stats.mean_coupon_discount == 0]

,mean_coupon_discount,mean_item_price
coupon_id,,
2,0.0,122.54
5,0.0,403.97
15,0.0,222.86
19,0.0,81.82
25,0.0,285.71
...,...,...
1111,0.0,118.08
1112,0.0,165.66
1113,0.0,125.97


In [109]:
coupon_stats.loc[coupon_stats.mean_coupon_discount == 0, 'mean_coupon_discount'] = coupon_stats.mean_coupon_discount.mean()

In [110]:
len(coupon_stats.loc[np.absolute(coupon_stats.mean_coupon_discount) > coupon_stats.mean_item_price])

0

## Add customer stats

In [111]:
transactions.sample(5)

,customer_id,item_id,quantity,coupon_discount,original_price,coupon_used
488625,450,17018,1,0.0,24.58,0
217317,1538,50925,1,0.0,142.12,0
1056418,1337,14115,1,0.0,142.12,0
1103844,711,37187,1,0.0,213.36,0
23316,1068,13665,1,0.0,12.11,0


In [112]:
pt_sums = pd.pivot_table(transactions, index='customer_id',
                         values=['quantity', 'original_price', 'coupon_discount', 'coupon_used'],
                         aggfunc={
                             'quantity': np.sum,
                             'original_price': lambda x: np.round(np.sum(x), decimals=2),
                             'coupon_discount': lambda x: np.round(np.sum(x), decimals=2),
                             'coupon_used': np.sum
                         })
pt_sums.reset_index(inplace=True)
pt_sums.rename(columns={
    'quantity': 'total_quantity_bought_by_cust',
    'original_price': 'total_price_paid_by_cust',
    'coupon_discount': 'total_discount_used_by_cust',
    'coupon_used': 'total_coupons_used_by_cust'}, inplace=True)
pt_sums

,customer_id,total_discount_used_by_cust,total_coupons_used_by_cust,total_price_paid_by_cust,total_quantity_bought_by_cust
0,1,-1832.94,78,103982.01,1048
1,2,-189.97,4,45360.60,419
2,3,-1329.46,53,61312.59,707
3,4,-17.81,1,30434.30,220
4,5,-90.83,2,91553.24,814
...,...,...,...,...,...
1577,1578,-613.86,29,72719.29,810
1578,1579,-145.45,6,132804.28,1162
1579,1580,0.00,0,59651.72,534
1580,1581,0.00,0,45356.97,541


In [113]:
pt_means = pd.pivot_table(transactions, index='customer_id',
                          values=['item_id', 'quantity', 'original_price', 'coupon_discount'],
                          aggfunc={
                              'item_id': lambda x: len(set(x)),
                              'quantity': lambda x: np.round(np.mean(x), decimals=2),
                              'original_price': lambda x: np.round(np.mean(x), decimals=2),
                              'coupon_discount': lambda x: np.round(np.mean(x), decimals=2)
                          })
pt_means.reset_index(inplace=True)
pt_means.rename(columns={
    'item_id': 'unique_items_bought_by_cust',
    'quantity': 'mean_quantity_bought_by_cust',
    'original_price': 'mean_selling_price_paid_by_cust',
    'coupon_discount': 'mean_discount_used_by_cust'}, inplace=True)
pt_means

,customer_id,mean_discount_used_by_cust,unique_items_bought_by_cust,mean_selling_price_paid_by_cust,mean_quantity_bought_by_cust
0,1,-1.75,463,99.22,1.00
1,2,-0.45,352,108.26,1.00
2,3,-1.89,406,86.97,1.00
3,4,-0.08,125,138.34,1.00
4,5,-0.11,490,115.60,1.03
...,...,...,...,...,...
1577,1578,-0.78,481,92.87,1.03
1578,1579,-0.13,639,114.29,1.00
1579,1580,0.00,422,112.76,1.01
1580,1581,0.00,390,89.82,1.07


In [114]:
cust_tran_stats = pd.merge(pt_means, pt_sums, on='customer_id')
cust_tran_stats

,customer_id,mean_discount_used_by_cust,unique_items_bought_by_cust,mean_selling_price_paid_by_cust,mean_quantity_bought_by_cust,total_discount_used_by_cust,total_coupons_used_by_cust,total_price_paid_by_cust,total_quantity_bought_by_cust
0,1,-1.75,463,99.22,1.00,-1832.94,78,103982.01,1048
1,2,-0.45,352,108.26,1.00,-189.97,4,45360.60,419
2,3,-1.89,406,86.97,1.00,-1329.46,53,61312.59,707
3,4,-0.08,125,138.34,1.00,-17.81,1,30434.30,220
4,5,-0.11,490,115.60,1.03,-90.83,2,91553.24,814
...,...,...,...,...,...,...,...,...,...
1577,1578,-0.78,481,92.87,1.03,-613.86,29,72719.29,810
1578,1579,-0.13,639,114.29,1.00,-145.45,6,132804.28,1162
1579,1580,0.00,422,112.76,1.01,0.00,0,59651.72,534
1580,1581,0.00,390,89.82,1.07,0.00,0,45356.97,541


In [115]:
if not os.path.exists('csv_4_db'):
    os.mkdir('csv_4_db')

In [138]:
coupons_items[['coupon_id', 'item_id', 'category']].sort_values('coupon_id').drop_duplicates()\
    .to_csv('csv_4_db/coupon_categories.csv', index=False)

In [141]:
coupon_stats['mean_coupon_discount'] = coupon_stats.mean_coupon_discount.apply(lambda x: np.round(x, decimals=2))
coupon_stats.reset_index().to_csv('csv_4_db/coupon_info.csv', index=False)

In [142]:
cust_stats = pd.merge(customers, cust_tran_stats, on='customer_id', how='left')
cust_stats.to_csv('csv_4_db/customer_info.csv', index=False)

,customer_id,age_range,marital_status,family_size,no_of_children,income_bracket,gender,mean_discount_used_by_cust,unique_items_bought_by_cust,mean_selling_price_paid_by_cust,mean_quantity_bought_by_cust,total_discount_used_by_cust,total_coupons_used_by_cust,total_price_paid_by_cust,total_quantity_bought_by_cust
0,1,70+,Married,2,0,4,M,-1.75,463,99.22,1.00,-1832.94,78,103982.01,1048
1,2,46-55,Married,2,0,3,F,-0.45,352,108.26,1.00,-189.97,4,45360.60,419
2,3,26-35,Single,1,0,4,M,-1.89,406,86.97,1.00,-1329.46,53,61312.59,707
3,4,46-55,Single,1,0,5,M,-0.08,125,138.34,1.00,-17.81,1,30434.30,220
4,5,70+,Married,2,0,4,F,-0.11,490,115.60,1.03,-90.83,2,91553.24,814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577,1578,46-55,Married,3,1,6,M,-0.78,481,92.87,1.03,-613.86,29,72719.29,810
1578,1579,46-55,Single,1,0,4,M,-0.13,639,114.29,1.00,-145.45,6,132804.28,1162
1579,1580,26-35,Married,2,0,5,M,0.00,422,112.76,1.01,0.00,0,59651.72,534
1580,1581,26-35,Married,3,1,1,M,0.00,390,89.82,1.07,0.00,0,45356.97,541


# Merge into one df for training - version 1

In [119]:
# transactions.sample(5)

In [120]:
# train_A = pd.merge(transactions, cust_stats, on='customer_id', how='left')
# train_A = pd.merge(train_A, items, on='item_id', how='left')

In [121]:
# train_A.info()

In [122]:
# train_A.original_price = np.round(train_A.original_price, decimals=2)

In [123]:
# train_A.drop(['customer_id', 'item_id', 'quantity', 'mean_item_price', 'mean_coupon_discount'], axis=1, inplace=True)
# train_A.shape

In [124]:
# train_A.drop_duplicates(inplace=True)
# train_A.shape

# Merge everything with train df - version 2

In [125]:
train = pd.read_csv('original_data/train.csv')
train.sample(5)

,id,campaign_id,coupon_id,customer_id,redemption_status
18752,30876,13,714,1174,0
63232,103879,13,21,275,0
50353,82563,27,822,787,0
4203,7015,8,725,520,0
40766,66929,8,14,520,0


In [126]:
train = pd.merge(train, cust_stats, on='customer_id', how='left')
train = pd.merge(train, coupon_stats, on='coupon_id', how='left')
train = pd.merge(train, coupons_items[['coupon_id', 'item_id', 'category']], on='coupon_id', how='left')
train

,id,campaign_id,coupon_id,customer_id,redemption_status,age_range,marital_status,family_size,no_of_children,income_bracket,...,mean_selling_price_paid_by_cust,mean_quantity_bought_by_cust,total_discount_used_by_cust,total_coupons_used_by_cust,total_price_paid_by_cust,total_quantity_bought_by_cust,mean_coupon_discount,mean_item_price,item_id,category
0,1,13,27,1053,0,46-55,Single,1,0,5,...,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,24775,Sport
1,1,13,27,1053,0,46-55,Single,1,0,5,...,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,14958,Women
2,1,13,27,1053,0,46-55,Single,1,0,5,...,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,40431,Men
3,1,13,27,1053,0,46-55,Single,1,0,5,...,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,20749,Women
4,1,13,27,1053,0,46-55,Single,1,0,5,...,164.25,1.08,-89.05,1,50918.77,335,-1.13,118.41,56860,Women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419922,128595,13,681,623,0,36-45,Married,2,0,3,...,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,16981,Men
6419923,128595,13,681,623,0,36-45,Married,2,0,3,...,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,44676,Girls
6419924,128595,13,681,623,0,36-45,Married,2,0,3,...,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,38641,Sport
6419925,128595,13,681,623,0,36-45,Married,2,0,3,...,106.05,1.13,0.00,0,75509.00,804,-1.96,169.80,34285,Women


In [127]:
train.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'age_range', 'marital_status', 'family_size', 'no_of_children',
       'income_bracket', 'gender', 'mean_discount_used_by_cust',
       'unique_items_bought_by_cust', 'mean_selling_price_paid_by_cust',
       'mean_quantity_bought_by_cust', 'total_discount_used_by_cust',
       'total_coupons_used_by_cust', 'total_price_paid_by_cust',
       'total_quantity_bought_by_cust', 'mean_coupon_discount',
       'mean_item_price', 'item_id', 'category'],
      dtype='object')

In [128]:
train.drop(['id', 'campaign_id', 'item_id'], axis=1, inplace=True)

In [129]:
train_with_ids = train
train_with_ids.drop_duplicates(inplace=True)
train_with_ids.columns

Index(['coupon_id', 'customer_id', 'redemption_status', 'age_range',
       'marital_status', 'family_size', 'no_of_children', 'income_bracket',
       'gender', 'mean_discount_used_by_cust', 'unique_items_bought_by_cust',
       'mean_selling_price_paid_by_cust', 'mean_quantity_bought_by_cust',
       'total_discount_used_by_cust', 'total_coupons_used_by_cust',
       'total_price_paid_by_cust', 'total_quantity_bought_by_cust',
       'mean_coupon_discount', 'mean_item_price', 'category'],
      dtype='object')

In [130]:
train.shape

(293483, 20)

In [131]:
train = train.drop(['coupon_id', 'customer_id'], axis=1)
train.drop_duplicates(inplace=True)
train.shape

(289477, 18)

# Encoding

In [132]:
def encode(df):
    encoder = LabelEncoder()
    
    df['age_range'] = encoder.fit_transform(df['age_range'])
    print('Age range encoding:')
    for i, c in enumerate(encoder.classes_):
        print(f'{i}: {c}')
        
    df['marital_status'] = encoder.fit_transform(df['marital_status'])
    print('Marital status encoding:')
    for i, c in enumerate(encoder.classes_):
        print(f'{i}: {c}')
        
    df['gender'] = encoder.fit_transform(df['gender'])
    print('Gender encoding:')
    for i, c in enumerate(encoder.classes_):
        print(f'{i}: {c}')
        
    df = pd.get_dummies(df, columns=['category'])
    return df

In [133]:
# train_A_encoded = encode(train_A)
train_encoded = encode(train)
train_with_ids_encoded = encode(train_with_ids)

Age range encoding:
0: 18-25
1: 26-35
2: 36-45
3: 46-55
4: 56-70
5: 70+
Marital status encoding:
0: Married
1: Single
Gender encoding:
0: F
1: M
Age range encoding:
0: 18-25
1: 26-35
2: 36-45
3: 46-55
4: 56-70
5: 70+
Marital status encoding:
0: Married
1: Single
Gender encoding:
0: F
1: M


## Save to files

In [134]:
dirname = 'prepped-data'
if not os.path.exists(dirname):
    os.mkdir(dirname)

In [135]:
train_with_ids.to_csv(os.path.join(dirname, 'train_with_ids.csv'), index=False)

In [136]:
# train_A_encoded.to_csv(os.path.join(dirname, 'train_A.csv'), index=False)
train_encoded.to_csv(os.path.join(dirname, 'train_B.csv'), index=False)